- Need to add an intro to motivate what you are doing before going into the steps."

We use the numpy modules so that we can easily create arrays and access math routines that operate in parallel over those arrays.

Next, we import the rebound module, which is an open-source python package that allows us to easily evaluate the gravitational interactions between an $N$ number of particles.  In our case, the particles will be the host star and the 5 planets in Kepler-62.

- In order to accurately input the Earth, Kepler star, and all of the kepler planets, we would need to add the components of each object. 
The "M" is mass, while the "e" is the code name for Earth. It is quite simple for the system to understand.
The input would look like this:
  M_e = 3.0035e-6

- A good code name for the Kepler star would be "star_K62." We will also input the mass, which looks like this:
  Mstar_K62 = 0.69

In [ ]:
import rebound
import numpy as np 

M_sun = 1. #Mass of the Sun in M_odot
M_e = 3.0035e-6 # Mass of the Earth in Solar masses (M_odot)

We define the initial states of the planets following the values for Case A in Quarles et al. (2020).  These values are given in the orbital elements (i.e., semimajor axis, eccentricity, orbital inclination, argument of pericenter, right ascension of ascending node, mean anomaly, and mass).  These values are given by:

| planet | $a$ (AU) | $e$ | $i$ (deg.) | $\omega$ (deg.) | $\Omega$ (deg.) | MA (deg.) | m ($M_e$) |
| ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ |
|  K62b | 0.0552802 | 0.07071068 | 0.883 | 278.13010 | 90.373788 | 74.589661 | 2.71 |

- Since we are more focused on the planets, we would need to add more components into the system. 
The components we need to add are semimajor axis, eccentricity, inclination, omega, Omega, mean anomaly, and the mass in comparison to the Sun. The input:
  K62b = [0.0552802,0.07071068,0.883,278.13010,90.373788,74.589661,2.71*M_e] #(semimajor axis, ecc, inclination, omega, Omega, Mean Anomaly, mass in M_sun)
  K62c = [0.0928576,0.08602325,0.560,54.462322,90.473841,327.80383578,0.128*M_e]
  K62d = [0.1195013,0.09486833,0.300,288.43494,90.002197,243.09602,4.94*M_e]
  K62e = [0.4263096,0.13000000,0.303,292.61986,90.303058,63.220181,4.05*M_e]
  K62f = [0.7176171,0.09433981,0.488,57.994616,90.478155,176.54639,3.57*M_e]

Each array containing the initial orbital parameters is stored in a Python list called `Pl_param`, which we use to iterate through common actions on each of the initial states (e.g., degrees to radians conversion).

```python
pl_param = [K62b,K62c,K62d,K62e,K62f]
```

- Pl_Param is an array that store all of the information of the objects into a neat archive. This is very productive instead of copying and pasting the information
many times again. All of the planets have code names for which the system can understand. The input looks like this:
  pl_param = [K62b,K62c,K62d,K62e,K62f]

A rebound simulation is instantiated (or created in memory), where we define the integrator and units assumed for the simulation.  We use the `whfast` integrator to simulate our system because it is the most optimal integration method (with respect to wall time) for solving nearly Keplerian-type problems and it preserves a low error in the total energy as the simulation progresses over long timescales.  In addition, the WHFast algorthim (Rein & Tamayo 2015) has been shown to a fast and unbiased version of the symplectic Wisdom-Holman integrator (Wisdom and Holman 1992). Our simulation is given in units of years, astronomical units (AU), and solar masses $M_\odot$.  Ultimately, we are evaluating the obliquity evolution and have chosen an initial condition that is stable for at least 10 Myr, such that no collisions or scattering events will occur.

```python
sim = rebound.Simulation()
sim.integrator = 'whfast'
sim.units = {'yr','Msun','AU'}
```

WHFast, a fast and unbiased symplectic Wisdom-Holman integrator. 
This integrator is well suited for integrations of planetary systems in which the planets stay roughly on their orbits. 
If close encounters and collisions occur, then WHFast is not the right integrator.

Next, we would need to add the names in which the units are labeled. For instance, we are doing Kepler 62. So, our input would look like this:

In [ ]:
sim.add(m=Mstar_K62)
#Don't do bottom sim.add, bc of below for loop
#Take lenght of pl_param (pl_param) and retrieves information of each particle 
for i in range(0,len(pl_param)):
    temp = pl_param[i]
    sim.add(m=temp[-1],a=temp[0],e=temp[1],inc=temp[2],omega=temp[3],Omega=temp[4],M=temp[5])
sim.move_to_com()